## Preprocessing

In [1]:

from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(['EIN', 'NAME'], axis=1)
df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning

df.APPLICATION_TYPE.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
application_values = df.APPLICATION_TYPE.value_counts()
application_values[application_values>500]
# application_values[application_values<500].index

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced

# use the variable name `application_types_to_replace`
application_types_to_replace = application_values[application_values<500].index


# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning

class_values = df['CLASSIFICATION'].value_counts()
class_values[class_values<1000].sum()

2261

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1


In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = class_values[class_values<1000].index
# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [12]:
#checking to see all the columns with unique number of values, so what to convert to get dummies
for i in df.columns:
  # print(i)
  print(df[i].value_counts())


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64
Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64
1    34294
0        5
Name: STATUS, dtype: int64
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+             

In [13]:
amt_values = df['ASK_AMT'].value_counts()
amt_values[amt_values>5]
# amt_values[amt_values<5].sum()


5000    25398
Name: ASK_AMT, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`

X_data = df.drop('IS_SUCCESSFUL', axis=1)
X_data.head()
y = df['IS_SUCCESSFUL']
y.head()

0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [15]:
X = pd.get_dummies(X_data)
X_data.shape
X.shape

(34299, 43)

In [16]:
# Split the preprocessed data into a training and testing dataset

X_train, X_test,y_train, y_test = train_test_split(X, y, random_state=10)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)

X_test_scaled = X_scaler.transform(X_test)



## Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4400      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 7,751
Trainable params: 7,751
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6002 - accuracy: 0.7009
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5676 - accuracy: 0.7307
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5588 - accuracy: 0.7298
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5542 - accuracy: 0.7334
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5522 - accuracy: 0.7338
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5506 - accuracy: 0.7347
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5491 - accuracy: 0.7342
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5480 - accuracy: 0.7346
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7349
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5466 - accura

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5570 - accuracy: 0.7277
Loss: 0.5569952130317688, Accuracy: 0.7276967763900757


In [23]:
# Redefining the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.


nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=43))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Third hidden layer
nn1.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Fourth hidden layer
nn1.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               4400      
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 30)                1530      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 11        
Total params: 11,301
Trainable params: 11,301
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model
fit_model = nn1.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5678 - accuracy: 0.7233
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5539 - accuracy: 0.7297
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5512 - accuracy: 0.7328
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5498 - accuracy: 0.7335
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5490 - accuracy: 0.7336
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5482 - accuracy: 0.7333
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5472 - accuracy: 0.7358
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5472 - accuracy: 0.7353
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5462 - accuracy: 0.7353
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5452 - accura

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5665 - accuracy: 0.7273
Loss: 0.5664546489715576, Accuracy: 0.7273469567298889


In [27]:
# Export our model to HDF5 file
from google.colab import files
nn.save('Deep_Learning_model.h5')
files.download('Deep_Learning_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>